# Bright By Text: Outgoing Messages Analysis

## Setup and Data Import

In [1]:
# Widen notebook

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import libraries necessary
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
from time import time
import datetime
from IPython.display import display
import re
%matplotlib inline
import psycopg2
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.3f}'.format

In [4]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [5]:
# Query to get archived outbound messages for 2017

sql = '''
    select
    om.message_id,
    om.created_at as outbound_message_created_at,
    om.body as outbound_message_body,
    om.partner_id as outbound_message_partner_id,
    mt.name as message_type,
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    p.name as partner_name,
    p.created_at as partner_created_date,
    p.is_active as partner_is_active,
    p_state.name as partner_state,
    c.children_count,
    c.dob_youngest_child,
    c.dob_oldest_child,
    msm.scheduled_message_id
    
from 
(
select *
    from archive.outbound_messages
    where extract(year from created_at) = 2017
) om
left join public.message_types as mt
on om.message_type_id = mt.id 
left join messages_scheduled_messages msm 
on om.message_id = msm.message_id 
left join public.v_subscribers as s
on om.subscriber_id = s.id
left join 
(
    select subscriber_id, count(*) as children_count, min(date_of_birth) as dob_youngest_child, max(date_of_birth) as dob_oldest_child
    from public.children 
    group by subscriber_id
) c 
on om.subscriber_id = c.subscriber_id
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.timezones t
on z.time_zone_id = t.id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join public.partners as p
on p.id = s.partner_id
left join public.partner_types as pt 
on pt.id = p.type_id
left join public.states as p_state
on p_state.id = p.state_id
'''
archived_messages_2017 = pd.read_sql_query(sql, conn)
conn = None

In [6]:
archived_messages_2017

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,children_count,dob_youngest_child,dob_oldest_child,scheduled_message_id
0,6e92ae11-f983-479f-8e9c-2b689368954d,2017-05-03 15:11:30.588685,Invalid zipcode. Kindly check your zipcode and...,00000000-0000-0000-0000-000000000002,signup,53e4abdd-308a-4cfc-a6fc-410c89a28d2d,2017-05-03 14:30:08.345377,2017-05-03 14:30:08.345377,NaT,AT&T Wireless,11096,Inwood,-5.000,America/New_York,New York,Text SignUp,activated,English,None,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,2015-06-06,2015-06-06,None
1,57c0ec0e-f812-4b45-a33a-213f4663f1c2,2017-05-09 16:34:15.603110,Is this zipcode correct? If yes reply with 'Y...,00000000-0000-0000-0000-000000000002,signup,f328ee46-398c-422f-aa55-4816caeae06e,2017-05-09 16:31:26.230154,2017-05-09 16:31:26.230154,NaT,"T-Mobile USA, Inc.",11219,Brooklyn,-5.000,America/New_York,New York,Text SignUp,activated,English,None,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,2014-06-23,2014-06-23,None
2,17d99d4d-137a-4317-b4d8-a63a4690fc93,2017-05-04 18:40:41.904659,"If you care for a young child, Bright by Text ...",f92494a8-bcce-473f-a56e-0ae2515c827e,signup,99915cfb-856b-4d59-8b94-0bd26d1342c4,2017-05-04 18:39:17.165132,2017-05-04 18:39:17.165132,NaT,"T-Mobile USA, Inc.",78251,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,None,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,2.000,2014-09-02,2016-09-03,None
3,ffb5de23-d828-4e3a-becf-8b10b80cefba,2017-05-04 18:45:12.364195,Unrecognized date of birth format. Please res...,f92494a8-bcce-473f-a56e-0ae2515c827e,signup,99915cfb-856b-4d59-8b94-0bd26d1342c4,2017-05-04 18:39:17.165132,2017-05-04 18:39:17.165132,NaT,"T-Mobile USA, Inc.",78251,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,None,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,2.000,2014-09-02,2016-09-03,None
4,bc56c5c2-c5b2-4a95-bfd5-a284661e7052,2017-05-04 18:39:17.856495,KLRN welcomes you to Bright by Text! Expect 2 ...,f92494a8-bcce-473f-a56e-0ae2515c827e,signup,99915cfb-856b-4d59-8b94-0bd26d1342c4,2017-05-04 18:39:17.165132,2017-05-04 18:39:17.165132,NaT,"T-Mobile USA, Inc.",78251,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,None,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,2.000,2014-09-02,2016-09-03,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518155,e47de3c4-7eb7-4344-b597-59b88c22490b,2017-12-30 22:15:45.432168,Time makes sense to your child in terms of her...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,6e44764b-7553-45b4-87b3-9d6375c3a793,2017-06-19 11:46:27.836526,2015-12-17 13:29:00.000000,NaT,"T-Mobile USA, Inc.",80030,Westminster,-7.000,America/Denver,Colorado,Sales Force,activated,English,None,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,2.000,2015-04-12,2015-04-13,None
1518156,7f132f43-3c40-4dca-a2ef-935c9027875c,2017-12-30 22:23:50.798022,"Sing the same song for specific activities, li...",d4d696af-37ff-427e-8035-2cb1bd630495,normal,127452b6-e5d5-456c-85a2-9470b740435d,2017-06-19 12:48:50.280428,2016-08-03 16:59:00.000000,NaT,"T-Mobile USA, Inc.",80247,Denver,-7.000,America/Denver,Colorado,Sales Force,activated,English,None,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,3.000,2015-01-02,2018-12-27,None
1518157,f34aa581-f57d-4bad-b796-51684185764c,2017-12-30 22:39:05.404773,"Singing Laundry: When you are doing laundry, s...",71718ec5-049d-47b9-80db-f9fbd0503152,normal,fef4ea4a-d491-474c-8762-6edb34530e73,2017-08-09 18:06:40.082440,2017-08-09 18:06:4

In [7]:
# Download the archived messages dataset for 2017

archived_messages_2017.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2017.csv', index=False)  

In [8]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [9]:
# Query to get archived outbound messages for 2018

sql = '''
    select
    om.message_id,
    om.created_at as outbound_message_created_at,
    om.body as outbound_message_body,
    om.partner_id as outbound_message_partner_id,
    mt.name as message_type,
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    p.name as partner_name,
    p.created_at as partner_created_date,
    p.is_active as partner_is_active,
    p_state.name as partner_state,
    c.children_count,
    c.dob_youngest_child,
    c.dob_oldest_child,
    msm.scheduled_message_id
    
from 
(
select *
    from archive.outbound_messages
    where extract(year from created_at) = 2018
) om
left join public.message_types as mt
on om.message_type_id = mt.id 
left join messages_scheduled_messages msm 
on om.message_id = msm.message_id 
left join public.v_subscribers as s
on om.subscriber_id = s.id
left join 
(
    select subscriber_id, count(*) as children_count, min(date_of_birth) as dob_youngest_child, max(date_of_birth) as dob_oldest_child
    from public.children 
    group by subscriber_id
) c 
on om.subscriber_id = c.subscriber_id
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.timezones t
on z.time_zone_id = t.id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join public.partners as p
on p.id = s.partner_id
left join public.partner_types as pt 
on pt.id = p.type_id
left join public.states as p_state
on p_state.id = p.state_id
'''
archived_messages_2018 = pd.read_sql_query(sql, conn)
conn = None

In [10]:
archived_messages_2018

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,children_count,dob_youngest_child,dob_oldest_child,scheduled_message_id
0,ea851042-0aa4-4b1d-97eb-2ec187638c34,2018-01-08 22:38:22.181992,Describe the things your child points to in bo...,3a18f949-d80f-481b-8cd8-3c73be63e67e,normal,4d02bf4b-c61b-4227-a5b2-b4a81eae335a,2017-12-12 12:32:42.113661,2017-12-12 12:32:42.113661,2019-08-12 17:00:08.865406,Onvoy/3 - Sybase365,90001,Los Angeles,-8.000,America/Los_Angeles,California,Web SignUp,deactivated,English,Age Out,United Way for Greater Austin,2018-08-28 20:57:33.540197,True,Texas,1.000,2017-12-12,2017-12-12,None
1,b23b4071-e7ae-40b1-822f-71905a77fbe3,2018-01-01 22:26:43.786828,Seeing patterns is an important pre-math skill...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,16d242b2-c6b6-4f06-9f81-78ad2c7f766e,2017-06-19 13:25:46.393823,2017-03-27 13:00:00.000000,NaT,"T-Mobile USA, Inc.",80204,Denver,-7.000,America/Denver,Colorado,Sales Force,activated,English,None,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,1.000,2013-11-29,2013-11-29,None
2,dc98843c-224c-4634-b762-3b23dca92ff7,2018-01-01 22:26:55.864001,Help your child learn by imitating you. Encour...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,52ee6736-3a59-4a56-8bdb-4be856ef79c1,2017-06-19 13:26:52.539459,2017-04-03 09:58:00.000000,NaT,AT&T Wireless,80033,Wheat Ridge,-7.000,America/Denver,Colorado,Sales Force,activated,English,None,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,1.000,2017-03-31,2017-03-31,None
3,79f32613-5735-4dd8-9206-84856cc823c8,2018-01-01 22:33:04.532669,"When children's curiosity is nurtured, so is t...",3a18f949-d80f-481b-8cd8-3c73be63e67e,normal,4d02bf4b-c61b-4227-a5b2-b4a81eae335a,2017-12-12 12:32:42.113661,2017-12-12 12:32:42.113661,2019-08-12 17:00:08.865406,Onvoy/3 - Sybase365,90001,Los Angeles,-8.000,America/Los_Angeles,California,Web SignUp,deactivated,English,Age Out,United Way for Greater Austin,2018-08-28 20:57:33.540197,True,Texas,1.000,2017-12-12,2017-12-12,None
4,5cb896cc-34e6-4067-b602-3f7959a38452,2018-01-01 22:33:04.996658,Writing starts with scribbling! Try this game ...,3a18f949-d80f-481b-8cd8-3c73be63e67e,normal,4d02bf4b-c61b-4227-a5b2-b4a81eae335a,2017-12-12 12:32:42.113661,2017-12-12 12:32:42.113661,2019-08-12 17:00:08.865406,Onvoy/3 - Sybase365,90001,Los Angeles,-8.000,America/Los_Angeles,California,Web SignUp,deactivated,English,Age Out,United Way for Greater Austin,2018-08-28 20:57:33.540197,True,Texas,1.000,2017-12-12,2017-12-12,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104461,f56f60fe-e435-4d8b-975a-771ae324a880,2018-12-28 17:34:07.051202,The Financial Empowerment Center helps you wit...,None,scheduled,b1e07f05-e263-40cb-ac73-a69484002a70,2017-11-23 00:08:36.017898,2017-11-23 00:08:36.017898,NaT,Verizon Wireless,78229,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,None,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,1.000,2015-08-15,2015-08-15,ca75334a-2148-40c7-a0bf-8740dab820ec
5104462,f56f60fe-e435-4d8b-975a-771ae324a880,2018-12-28 17:34:10.823055,The Financial Empowerment Center helps you wit...,None,scheduled,3e42e7e7-1021-4955-bb7b-d3ceb099a99b,2018-07-21 15:48:05.519417,2018-07-21 15:48:05.519417,NaT,"Sprint Spectrum, L.P.",78251,San Antonio,-6.000,America/Chicago,Texas,Web SignUp,activated,English,None,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,1.000,2015-11-15,2015-11-15,ca75334a-2148-40c7-a0bf-8740dab820ec
5104463,f56f60fe-e435-4d8b-975a-771ae324a880,2018-12-28 17:34:07.083416,The Financial Empowerment Center helps you wit..

In [11]:
# Download the archived messages dataset for 2018

archived_messages_2018.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2018.csv', index=False)  

In [12]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [13]:
# Query to get archived outbound messages for 1H 2019

sql = '''
    select
    om.message_id,
    om.created_at as outbound_message_created_at,
    om.body as outbound_message_body,
    om.partner_id as outbound_message_partner_id,
    mt.name as message_type,
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    p.name as partner_name,
    p.created_at as partner_created_date,
    p.is_active as partner_is_active,
    p_state.name as partner_state,
    c.children_count,
    c.dob_youngest_child,
    c.dob_oldest_child,
    msm.scheduled_message_id
    
from 
(
select *
    from archive.outbound_messages
    where extract(year from created_at) = 2019 
      and extract(month from created_at) <= 6
) om
left join public.message_types as mt
on om.message_type_id = mt.id 
left join messages_scheduled_messages msm 
on om.message_id = msm.message_id 
left join public.v_subscribers as s
on om.subscriber_id = s.id
left join 
(
    select subscriber_id, count(*) as children_count, min(date_of_birth) as dob_youngest_child, max(date_of_birth) as dob_oldest_child
    from public.children 
    group by subscriber_id
) c 
on om.subscriber_id = c.subscriber_id
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.timezones t
on z.time_zone_id = t.id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join public.partners as p
on p.id = s.partner_id
left join public.partner_types as pt 
on pt.id = p.type_id
left join public.states as p_state
on p_state.id = p.state_id
'''
archived_messages_2019_1h = pd.read_sql_query(sql, conn)
conn = None

In [14]:
archived_messages_2019_1h

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,children_count,dob_youngest_child,dob_oldest_child,scheduled_message_id
0,71f710a6-0ac1-4d86-8d00-aa5d3d0e538b,2019-04-11 16:31:54.071518,BBT provides resources for children prenatal t...,71718ec5-049d-47b9-80db-f9fbd0503152,signup,95c68cc4-d62e-4951-b2b0-94b6e4951974,2019-04-11 16:29:49.966083,2019-04-11 16:32:14.249942,NaT,Cricket Wireless - ATT - SVR,93710,Fresno,-8.000,America/Los_Angeles,California,Text SignUp,activated,English,None,"ValleyPBS - Fresno, CA",2018-08-28 20:57:03.730535,True,California,1.000,2016-09-24,2016-09-24,None
1,d246e81a-402b-4151-ae25-3d0f425eb74e,2019-04-11 14:48:23.417284,We don't recognize this zip code. Please check...,71718ec5-049d-47b9-80db-f9fbd0503152,signup,446556b8-9fd0-4c1f-91b4-e4216e8e473a,2019-04-11 14:30:42.240335,2019-04-11 14:47:43.708584,NaT,"Sprint Spectrum, L.P.",93230,Hanford,-8.000,America/Los_Angeles,California,Text SignUp,activated,English,None,"ValleyPBS - Fresno, CA",2018-08-28 20:57:03.730535,True,California,1.000,2011-10-25,2011-10-25,None
2,ea8008c1-373d-447c-bb60-cd89c87bce96,2019-04-11 04:41:23.301082,Reply Y if 86305 is correct or re-send your zi...,00000000-0000-0000-0000-000000000001,signup,054c0bb6-0385-4dc0-a50c-b10a96c0548a,2019-04-11 04:38:50.311933,2019-04-11 04:41:06.916667,NaT,Verizon Wireless,86305,Prescott,-7.000,America/Phoenix,Arizona,Text SignUp,activated,English,None,ArizonaPBS,2018-03-25 21:32:18.027322,True,Arizona,1.000,2017-03-17,2017-03-17,None
3,eb9a3d3d-d9ee-42a5-8b82-bbfba4f8e9af,2019-04-19 03:00:14.131447,We want to learn more about you! Please fill o...,d4d696af-37ff-427e-8035-2cb1bd630495,survey,80ab2d95-06bf-4f64-963f-d935cf92fede,2019-03-25 18:53:50.387224,2019-03-25 18:53:50.387224,2019-08-19 18:44:46.334876,Verizon Wireless,80022,Commerce City,-7.000,America/Denver,Colorado,Text SignUp,deactivated,English,STOP,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,1.000,2019-03-12,2019-03-12,None
4,ea8008c1-373d-447c-bb60-cd89c87bce96,2019-04-25 15:04:22.865104,Reply Y if 28083 is correct or re-send your zi...,97f6bf86-bbab-4e74-b3a9-3940bbc340a9,signup,d6dc7b09-5916-444b-b825-7f36e9312a64,2019-04-25 13:59:53.439772,2019-04-25 15:03:38.652131,NaT,"Sprint Spectrum, L.P.",28083,Kannapolis,-5.000,America/New_York,North Carolina,Text SignUp,activated,English,None,UNC-TV- North Carolina,2018-08-28 21:01:13.109946,True,North Carolina,1.000,2019-01-12,2019-01-12,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3013494,06092af5-b574-4fbc-b6f1-a623b02a6a5c,2019-01-28 17:31:43.337831,Parents and Caregivers! These resources will i...,None,scheduled,fd5bed6e-d2b9-4616-91fb-f992b6bd0e53,2018-05-22 14:02:02.456168,2019-10-16 15:27:23.502832,NaT,Verizon Wireless,84401,Ogden,-7.000,America/Denver,Utah,Text SignUp,activated,English,None,United Way of Northern Utah - Ogden,2018-08-28 21:02:34.021621,True,Utah,3.000,2011-02-23,2014-10-06,dace278e-6a7e-4549-aa6e-7a90f0b2887d
3013495,06092af5-b574-4fbc-b6f1-a623b02a6a5c,2019-01-28 17:31:46.290935,Parents and Caregivers! These resources will i...,None,scheduled,b7fc9b64-709f-4bae-b7f5-1106a485a92b,2018-04-27 14:34:19.602987,2018-04-27 14:34:19.602987,NaT,Verizon Wireless,84302,Brigham City,-7.000,America/Denver,Utah,Text SignUp,activated,English,None,United Way of Northern Utah - Ogden,2018-08-28 21:02:34.021621,True,Utah,1.000,2014-03-31,2014-03-31,dace278e-6a7e-4549-aa6e-7a90f0b2887d
3013496,06092af5-b574-4fbc-b6f1-a623b02a6a5c,2019-01-28 17:31:47.202758,Parents and Caregivers! These resources will i...,None,scheduled,6f048348-4f4b-42c

In [15]:
# Download the archived messages dataset for 2019 1H

archived_messages_2019_1h.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2019_1h.csv', index=False)  

In [16]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [17]:
# Query to get archived outbound messages for 2H 2019

sql = '''
    select
    om.message_id,
    om.created_at as outbound_message_created_at,
    om.body as outbound_message_body,
    om.partner_id as outbound_message_partner_id,
    mt.name as message_type,
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    p.name as partner_name,
    p.created_at as partner_created_date,
    p.is_active as partner_is_active,
    p_state.name as partner_state,
    c.children_count,
    c.dob_youngest_child,
    c.dob_oldest_child,
    msm.scheduled_message_id
    
from 
(
select *
    from archive.outbound_messages
    where extract(year from created_at) = 2019 
      and extract(month from created_at) > 6
) om
left join public.message_types as mt
on om.message_type_id = mt.id 
left join messages_scheduled_messages msm 
on om.message_id = msm.message_id 
left join public.v_subscribers as s
on om.subscriber_id = s.id
left join 
(
    select subscriber_id, count(*) as children_count, min(date_of_birth) as dob_youngest_child, max(date_of_birth) as dob_oldest_child
    from public.children 
    group by subscriber_id
) c 
on om.subscriber_id = c.subscriber_id
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.timezones t
on z.time_zone_id = t.id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join public.partners as p
on p.id = s.partner_id
left join public.partner_types as pt 
on pt.id = p.type_id
left join public.states as p_state
on p_state.id = p.state_id
'''
archived_messages_2019_2h = pd.read_sql_query(sql, conn)
conn = None

In [18]:
archived_messages_2019_2h

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,children_count,dob_youngest_child,dob_oldest_child,scheduled_message_id
0,356f0eca-f6ed-44f2-8ab3-89fb919f0286,2019-07-03 02:59:09.057012,Louisiana Public Broadcasting welcomes you to ...,0cf81fa0-fd85-44da-8702-d6e0eeb4fc8d,signup,da37347f-e2e8-4d19-ac99-9905cea6404a,2019-07-03 02:59:08.814264,2019-07-03 03:03:34.103855,2019-08-04 16:01:01.484052,Verizon Wireless,70769,Prairieville,-6.000,America/Chicago,Louisiana,Text SignUp,deactivated,English,STOP,Louisiana Public Broadcasting,2019-04-17 19:49:15.007478,True,Louisiana,1.000,2016-12-21,2016-12-21,None
1,39397597-c1a7-4218-9622-864380247ce0,2019-07-03 15:00:14.264828,Remember making forts and playing in cardboard...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,48405053-41fb-4750-b866-d87b6d426162,2019-01-03 22:29:20.341875,2019-01-03 22:29:20.341875,NaT,Verizon Wireless,14020,Batavia,-5.000,America/New_York,New York,Text SignUp,activated,English,None,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,1.000,2017-12-27,2017-12-27,None
2,a13fae68-63e5-44fd-8b47-23fb4b9f23bc,2019-07-03 15:00:15.266724,Use sunscreen every time your child is in the ...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,8941c36d-f05b-4edf-970f-fa0584f19b72,2018-06-24 22:38:51.605720,2018-06-24 22:38:51.605720,NaT,AT&T Wireless,11222,Brooklyn,-5.000,America/New_York,New York,Text SignUp,activated,English,None,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,1.000,2018-02-02,2018-02-02,None
3,87db92b3-7247-4f32-a103-3f647652901b,2019-07-03 15:00:16.019415,Your baby is already 10 months old! Growing up...,00000000-0000-0000-0000-000000000002,normal,d536b15e-ddb3-4012-a83e-fe4913569b4a,2019-07-03 11:31:44.173594,2019-07-03 11:33:42.693003,NaT,AT&T Wireless,10019,New York,-5.000,America/New_York,New York,Text SignUp,activated,English,None,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,2018-08-27,2018-08-27,None
4,957f1d7f-2ec8-400a-bafa-c308648e483e,2019-07-03 15:00:16.256914,Snack time is a great opportunity to get more ...,00000000-0000-0000-0000-000000000002,normal,5a8ccc15-9ef4-4637-967f-cf5513bf8172,2017-06-24 15:30:18.083001,2017-06-24 15:30:18.083001,2019-07-20 15:01:12.022771,"Sprint Spectrum, L.P.",12401,Kingston,-5.000,America/New_York,New York,Text SignUp,deactivated,English,STOP,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,2016-06-10,2016-06-10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4359035,21e1ee17-38c9-4338-a5a1-77acc0706345,2019-12-31 19:01:21.142854,Talk numbers every day to build your child’s m...,4090bf1f-7d21-40d3-bc70-298f367f61fc,normal,e047eeff-0459-4e2d-83e9-b30841a2b59a,2019-09-06 01:08:32.972570,2019-09-06 01:24:18.736755,NaT,Verizon Wireless,89108,Las Vegas,-8.000,America/Los_Angeles,Nevada,Text SignUp,activated,English,None,Vegas PBS,2019-04-19 15:17:56.225879,True,Nevada,1.000,2016-07-19,2016-07-19,None
4359036,7c897916-ee61-470c-8b0c-6dd1e7c11c15,2019-12-31 19:01:21.143040,You don't need expensive supplies to stock a c...,4090bf1f-7d21-40d3-bc70-298f367f61fc,normal,959c610c-de6e-45bc-b49f-9037d135e139,2019-06-02 23:02:08.873069,2019-06-02 23:46:12.200382,NaT,AT&T Wireless,89074,Henderson,-8.000,America/Los_Angeles,Nevada,Text SignUp,activated,English,None,Vegas PBS,2019-04-19 15:17:56.225879,True,Nevada,1.000,2016-03-26,2016-03-26,None
4359037,6222d714-b901-423e-8394-26e1a1730af3,2019-12-31 19:01:21.143822,Time-outs can be a good way to deal with misbe...,4090bf1f-7d21-40d3-bc70-298f367f61fc,normal,74701ed6-8cb8-4d03-ac7f-ad7ecec75d39,2019-11-

In [19]:
# Download the archived messages dataset for 2019 2H

archived_messages_2019_2h.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2019_2h.csv', index=False)  

In [24]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [25]:
# Query to get archived outbound messages for 2020

sql = '''
select
    om.message_id,
    om.created_at as outbound_message_created_at,
    om.body as outbound_message_body,
    om.partner_id as outbound_message_partner_id,
    mt.name as message_type,
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    p.name as partner_name,
    p.created_at as partner_created_date,
    p.is_active as partner_is_active,
    p_state.name as partner_state,
    c.children_count,
    c.dob_youngest_child,
    c.dob_oldest_child,
    msm.scheduled_message_id
    
from 
(
select *
    from archive.outbound_messages
    where extract(year from created_at) = 2020
) om
left join public.message_types as mt
on om.message_type_id = mt.id 
left join messages_scheduled_messages msm 
on om.message_id = msm.message_id 
left join public.v_subscribers as s
on om.subscriber_id = s.id
left join 
(
    select subscriber_id, count(*) as children_count, min(date_of_birth) as dob_youngest_child, max(date_of_birth) as dob_oldest_child
    from public.children 
    group by subscriber_id
) c 
on om.subscriber_id = c.subscriber_id
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.timezones t
on z.time_zone_id = t.id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join public.partners as p
on p.id = s.partner_id
left join public.partner_types as pt 
on pt.id = p.type_id
left join public.states as p_state
on p_state.id = p.state_id
'''
archived_messages_2020 = pd.read_sql_query(sql, conn)
conn = None

In [26]:
# Download the archived messages dataset for 2020

archived_messages_2020.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_archived_messages_2020.csv', index=False)  

In [27]:
messages = pd.concat([archived_messages_2017, archived_messages_2018, archived_messages_2019_1h, archived_messages_2019_2h, archived_messages_2020])

In [28]:
# Download the full messages dataset

messages.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_messages_2017_2020.csv', index=False)  

Note that this table only includes outbound messages, and does not include inbound messages.

## Query to get scheduled message tags

In [29]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [30]:
sql = '''
select sm.id as scheduled_message_id,
smt.name as scheduled_message_tag
from scheduled_messages sm 
left join scheduled_message_to_tags smtt 
on sm.id = smtt.scheduled_message_id 
left join scheduled_message_tags smt 
on smtt.scheduled_message_tag_id = smt.id 
'''
scheduled_message_tags = pd.read_sql_query(sql, conn)
conn = None

In [31]:
# Download the scheduled messages tags dataset

scheduled_message_tags.to_csv(r'/Users/lisafan/Desktop/BBT/scheduled_message_tags.csv', index=False)  